In [187]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd
from IPython.display import display, Markdown, Latex
from scipy.sparse.linalg import svds
import numpy as np

In [188]:
df_train = pd.read_csv('../Preprocessed Data/cleaned_dataset_train.csv')
df_test = pd.read_csv('../Preprocessed Data//cleaned_dataset_test.csv')
#df = pd.read_csv('../cleaned_dataset.csv')


In [189]:
X_train = df_train['comment'].fillna(" ").tolist()
Y_train = df_train['insult'].tolist()

X_test = df_test['comment'].fillna(" ").tolist()
Y_test = df_test['insult'].tolist()

vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)

X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)


In [195]:
u_train, s_train, v_train = svds(X_train)
u_test, s_test, v_test = svds(X_test)
print(u_train.shape, ' ', s_train.shape, ' ', v_train.shape)



(3947, 6)   (6,)   (6, 14818)


In [196]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C = 3, class_weight="balanced",warm_start=True)
clf.fit(u_train,Y_train)

LogisticRegression(C=3, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=True)

In [197]:
predictions = clf.predict_proba(u_test)

In [198]:
# count = 0
# for prediction in range(len(predictions)):
#     if predictions[prediction][1] == Y_test[prediction]:
#         count += 1
# acc = count/len(predictions)
final = []
for prediction in predictions:
    final.append(prediction[1])
    
acc = 0
aoc = metrics.roc_auc_score(Y_test,final)
data = "**AUC Score** : " + str(aoc) 
display(Markdown(data))       

**AUC Score** : 0.727342144947058

In [194]:
predictions = clf.predict(u_test)  
from sklearn.metrics import precision_recall_fscore_support

from sklearn.metrics import classification_report 

precision, recall, f_score, _ = precision_recall_fscore_support(Y_test, predictions)

print('Precision: ', precision, '  Recall: ', recall, '   F-score: ', f_score)

print(classification_report(Y_test, predictions))

Precision:  [0.78774423 0.5443038 ]   Recall:  [0.90788127 0.31024531]    F-score:  [0.84355682 0.39522059]
             precision    recall  f1-score   support

          0       0.79      0.91      0.84      1954
          1       0.54      0.31      0.40       693

avg / total       0.72      0.75      0.73      2647

